In [87]:
import os

import pandas as pd
from gspread_pandas import Spread, Client

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
dir_item = '/home/jake/Documents/News_Item/'
dir_proj = dir_item + 'uniqlo/'
dir_tableau = dir_proj + 'tableau_data/'

if not os.path.exists(dir_tableau):
    os.makedirs(dir_tableau)

In [20]:
def round_money_only_hundred_k(row):
    won = ['롯데', '신한', '하나', '현대', 'BC']
    card = row['카드사']
    brand = row['브랜드']
    try:
        sales = float(row['매출액'])
    except ValueError as e:
        print(row)
        raise ValueError(e)
    abs_sales = abs(sales)
    
    if (card not in won) or (card == '하나' and brand == '전체 신판 매출'):
        return sales / 100
    
    if pd.isna(sales):
        result = 0
    else:
        result = sales / 1000000
    return result / 100

In [28]:
def get_sales(df, dates):
    sales = df.melt(id_vars=['카드사', '브랜드'], value_vars=dates, var_name='기간', value_name='매출액')
    sales['매출액'] = sales['매출액'].str.strip()
    sales['매출액'] = sales['매출액'].str.replace(',','')
    sales['매출액_억원'] = sales.apply(round_money_only_hundred_k, axis=1)
    sales['브랜드'].replace('전체매출', '전체 신판 매출', inplace=True)
    sales['브랜드'].replace('브랜드 매출', '브랜드 합계', inplace=True)
    sales['브랜드'].replace(' 브랜드매출', '브랜드 합계', inplace=True)
    sales['브랜드'].replace('총합계', '브랜드 합계', inplace=True)

    dhc = sales.loc[sales['브랜드']=='DHC'].copy()
    sales = sales.pivot_table(index=['카드사', '기간'], columns='브랜드', values='매출액_억원')
    sales['DHC'].fillna(0, inplace=True)
    sales.reset_index(inplace=True)
    idx_sinhan = sales['카드사']=='신한'
    sales.loc[idx_sinhan, '브랜드 합계'] = sales.loc[idx_sinhan, '브랜드 합계'] - sales.loc[idx_sinhan, 'DHC']
    sales.drop('DHC', axis=1, inplace=True)
    idx_hana = sales['카드사']=='하나'
    sales.loc[idx_hana, '브랜드 합계'] = sales.loc[idx_hana, '유니클로'] + sales.loc[idx_hana, 'ABC마트'] + sales.loc[idx_hana, '무인양품']
    return sales, dhc

# Load spread

In [4]:
spread = Spread('일본브랜드_신용카드매출')

# sales_2019

In [5]:
sheet_index=0
spread.open_sheet(sheet_index)
df_2019 = spread.sheet_to_df(index=0, sheet=sheet_index)
df_2019.head()

,카드사,브랜드,20190624,20190701,20190708,20190715,20190722
0,KB국민,ABC마트,445,437,405,396,376
1,KB국민,유니클로,702,743,419,318,205
2,KB국민,무인양품,79,84,62,48,36
3,KB국민,전체매출,"2,063,786","1,806,097","1,963,796","1,819,302","2,132,572"
4,KB국민,브랜드 합계,"1,227","1,263",886,763,617


In [29]:
dates_2019 = ['20190624', '20190701', '20190708', '20190715', '20190722',]

sales_2019, dhc_2019 = get_sales(df_2019, dates_2019)
sales_2019

브랜드,카드사,기간,ABC마트,무인양품,브랜드 합계,유니클로,전체 신판 매출
0,BC,20190624,8.935725,1.464822,24.018441,13.617894,34023.207822
1,BC,20190701,9.602322,1.379833,23.532648,12.550494,29437.467374
2,BC,20190708,8.457011,0.943043,16.701093,7.301039,30722.613925
3,BC,20190715,8.511986,0.706640,14.552259,5.333633,30422.027463
4,BC,20190722,7.200970,0.550983,11.798662,4.046710,36715.413466
5,KB국민,20190624,4.450000,0.790000,12.270000,7.020000,20637.860000
6,KB국민,20190701,4.370000,0.840000,12.630000,7.430000,18060.970000
7,KB국민,20190708,4.050000,0.620000,8.860000,4.190000,19637.960000
8,KB국민,20190715,3.960000,0.480000,7.630000,3.180000,18193.020000
9,KB국민,20190722,3.760000,0.360000,6.170000,2.050000,21325.720000


신한: 전체 신판 매출 없음  
신한: DHC 보유  
하나: 브랜드 합계 없음  
원 단위: 롯데, 신한, 하나, 현대, BC  
하나: 전체 신판 매출은 원단위 아닌듯

In [30]:
sales_2019.columns
sales_2019

브랜드,카드사,기간,ABC마트,무인양품,브랜드 합계,유니클로,전체 신판 매출
0,BC,20190624,8.935725,1.464822,24.018441,13.617894,34023.207822
1,BC,20190701,9.602322,1.379833,23.532648,12.550494,29437.467374
2,BC,20190708,8.457011,0.943043,16.701093,7.301039,30722.613925
3,BC,20190715,8.511986,0.706640,14.552259,5.333633,30422.027463
4,BC,20190722,7.200970,0.550983,11.798662,4.046710,36715.413466
5,KB국민,20190624,4.450000,0.790000,12.270000,7.020000,20637.860000
6,KB국민,20190701,4.370000,0.840000,12.630000,7.430000,18060.970000
7,KB국민,20190708,4.050000,0.620000,8.860000,4.190000,19637.960000
8,KB국민,20190715,3.960000,0.480000,7.630000,3.180000,18193.020000
9,KB국민,20190722,3.760000,0.360000,6.170000,2.050000,21325.720000


In [19]:
sales_2019['카드사'].value_counts()

하나      5
BC      5
신한      5
롯데      5
현대      5
삼성      5
KB국민    5
Name: 카드사, dtype: int64

In [8]:
sales_2019.loc[sales_2019['기간']=='20190624',]

브랜드,카드사,기간,ABC마트,무인양품,브랜드 합계,유니클로,전체 신판 매출
0,BC,20190624,8.090305e+09,1.040604e+09,2.040001e+11,1.030601e+11,3.040002e+17
5,KB국민,20190624,4.040500e+03,7.090000e+01,1.020207e+05,7.000200e+03,2.000603e+11
10,롯데,20190624,2.060602e+09,5.010305e+07,8.040809e+09,5.030103e+09,9.060600e+15
15,삼성,20190624,4.000000e+03,9.010000e+01,1.020600e+05,7.060900e+03,1.080108e+11
20,신한,20190624,6.020406e+09,1.010606e+09,1.080005e+11,1.000603e+11,NaN
25,하나,20190624,5.090600e+09,1.000100e+09,1.117120e+10,5.080500e+09,1.020801e+11
30,현대,20190624,4.000601e+09,7.010106e+07,1.040005e+11,9.020601e+09,1.080508e+17


7개 카드사, 다른 카드사는 없는지?  
=> 8개 카드사인데 우리카드는 BC카드에 포함

In [37]:
def get_sum(sales):
    cols_brand = ['유니클로', 'ABC마트', '무인양품', '브랜드 합계']
    summ = sales[['기간'] + cols_brand].groupby('기간').sum()
    summ.reset_index(inplace=True)
    summ['기간'] = pd.to_datetime(summ['기간'])
    return summ

sum_2019 = get_sum(sales_2019)
sum_2019

브랜드,기간,유니클로,ABC마트,무인양품,브랜드 합계
0,2019-06-24,59.389386,36.316333,6.566196,102.298383
1,2019-07-01,56.696858,35.868507,5.885056,98.470555
2,2019-07-08,32.751962,33.163628,4.538853,70.481827
3,2019-07-15,25.129224,32.399574,3.200002,60.794089
4,2019-07-22,17.733289,29.346885,2.710919,49.805632


# Export: sum_2019

In [89]:
# sum_2019.to_csv(dir_tableau + 'sum_2019.tsv', sep='\t', index=False)

In [50]:
def print_4_week_diff(summ):
    tmp = summ.iloc[0, 1:] - summ.iloc[4, 1:]
    print(tmp)
    print(tmp / summ.iloc[0, 1:] * 100)

    
print_4_week_diff(sum_2019)

브랜드
유니클로      41.6561
ABC마트     6.96945
무인양품      3.85528
브랜드 합계    52.4928
dtype: object
브랜드
유니클로      70.1406
ABC마트     19.1909
무인양품      58.714 
브랜드 합계    51.3134
dtype: object


In [51]:
def get_summ_diff(summ):
    return summ.iloc[::, 1:] - summ.iloc[::, 1:].shift(1)


get_summ_diff(sum_2019)

브랜드,유니클로,ABC마트,무인양품,브랜드 합계
0,NaN,NaN,NaN,NaN
1,-2.692529,-0.447826,-0.681140,-3.827828
2,-23.944895,-2.704879,-1.346203,-27.988728
3,-7.622738,-0.764054,-1.338851,-9.687738
4,-7.395935,-3.052689,-0.489083,-10.988457


# sales_2018

In [10]:
sheet_index=1
spread.open_sheet(sheet_index)
df_2018 = spread.sheet_to_df(index=0, sheet=sheet_index)
df_2018.head()

,카드사,브랜드,20180624,20180701,20180708,20180715,20180722
0,KB국민,ABC마트,400,382,419,404,386
1,KB국민,유니클로,583,621,640,717,670
2,KB국민,무인양품,66,70,65,73,71
3,KB국민,전체매출,"1,846,883","1,635,637","1,843,287","1,690,585","2,073,277"
4,KB국민,브랜드 합계,"1,049","1,073","1,124","1,194","1,127"


In [32]:
dates_2018 = ['20180624', '20180701', '20180708', '20180715', '20180722',]

sales_2018, dhc_2018 = get_sales(df_2018, dates_2018)
sales_2018

브랜드,카드사,기간,ABC마트,무인양품,브랜드 합계,유니클로,전체 신판 매출
0,BC,20180624,9.343524,1.255122,23.088166,12.489520,32262.382071
1,BC,20180701,9.762218,1.196422,23.748956,12.790316,27857.048055
2,BC,20180708,10.448573,1.245766,24.895531,13.201192,30648.546210
3,BC,20180715,9.437479,1.174330,24.633709,14.021900,30702.720138
4,BC,20180722,9.066076,1.114661,22.931632,12.750895,34536.529018
5,KB국민,20180624,4.000000,0.660000,10.490000,5.830000,18468.830000
6,KB국민,20180701,3.820000,0.700000,10.730000,6.210000,16356.370000
7,KB국민,20180708,4.190000,0.650000,11.240000,6.400000,18432.870000
8,KB국민,20180715,4.040000,0.730000,11.940000,7.170000,16905.850000
9,KB국민,20180722,3.860000,0.710000,11.270000,6.700000,20732.770000


In [33]:
# sales_2018['브랜드'].value_counts()
sales_2018.columns
sales_2018.head()

브랜드,카드사,기간,ABC마트,무인양품,브랜드 합계,유니클로,전체 신판 매출
0,BC,20180624,9.343524,1.255122,23.088166,12.489520,32262.382071
1,BC,20180701,9.762218,1.196422,23.748956,12.790316,27857.048055
2,BC,20180708,10.448573,1.245766,24.895531,13.201192,30648.546210
3,BC,20180715,9.437479,1.174330,24.633709,14.021900,30702.720138
4,BC,20180722,9.066076,1.114661,22.931632,12.750895,34536.529018


In [49]:
sum_2018 = get_sum(sales_2018)
sum_2018

브랜드,기간,유니클로,ABC마트,무인양품,브랜드 합계
0,2018-06-24,46.092345,32.969550,5.854442,84.944416
1,2018-07-01,53.864734,33.489248,5.611049,93.022912
2,2018-07-08,55.527404,35.554137,5.581648,96.691126
3,2018-07-15,61.373411,33.834724,5.702784,100.972077
4,2018-07-22,56.471693,33.320317,5.504085,95.322482


# Export: sum_2018

In [90]:
# sum_2018.to_csv(dir_tableau + 'sum_2018.tsv', sep='\t', index=False)

In [111]:
def add_dt_repr(x):
    return '2019-' + str(x.month) + '-' + str(x.day)


tmp_2019 = sum_2019.copy()
tmp_2018 = sum_2018.copy()

tmp_2019.columns = [c + '_2019' for c in tmp_2019.columns]
tmp_2018.columns = [c + '_2018' for c in tmp_2018.columns]

tmp_2019['dt_repr'] = tmp_2019['기간_2019'].apply(add_dt_repr)
tmp_2018['dt_repr'] = tmp_2018['기간_2018'].apply(add_dt_repr)

sum_both = tmp_2019.merge(tmp_2018, on='dt_repr', how='left')
sum_both

,기간_2019,유니클로_2019,ABC마트_2019,무인양품_2019,브랜드 합계_2019,dt_repr,기간_2018,유니클로_2018,ABC마트_2018,무인양품_2018,브랜드 합계_2018
0,2019-06-24,59.389386,36.316333,6.566196,102.298383,2019-6-24,2018-06-24,46.092345,32.969550,5.854442,84.944416
1,2019-07-01,56.696858,35.868507,5.885056,98.470555,2019-7-1,2018-07-01,53.864734,33.489248,5.611049,93.022912
2,2019-07-08,32.751962,33.163628,4.538853,70.481827,2019-7-8,2018-07-08,55.527404,35.554137,5.581648,96.691126
3,2019-07-15,25.129224,32.399574,3.200002,60.794089,2019-7-15,2018-07-15,61.373411,33.834724,5.702784,100.972077
4,2019-07-22,17.733289,29.346885,2.710919,49.805632,2019-7-22,2018-07-22,56.471693,33.320317,5.504085,95.322482


# Export: sum_both

In [112]:
# sum_both.to_csv(dir_tableau + 'sum_both.tsv', sep='\t', index=False)

In [60]:
(61.4 - 46.1) / 46.1

0.33188720173535785

In [61]:
(35.6-33) / 33

0.07878787878787884

In [53]:
tmp = sum_2019 - sum_2018
tmp

브랜드,기간,유니클로,ABC마트,무인양품,브랜드 합계
0,365 days,13.297042,3.346783,0.711754,17.353967
1,365 days,2.832124,2.379259,0.274007,5.447643
2,365 days,-22.775441,-2.390509,-1.042795,-26.209300
3,365 days,-36.244187,-1.435150,-2.502782,-40.177988
4,365 days,-38.738403,-3.973432,-2.793167,-45.516850


In [56]:
tmp.iloc[::, 1:] / sum_2018.iloc[::, 1:] * 100

브랜드,유니클로,ABC마트,무인양품,브랜드 합계
0,28.848699,10.151134,12.157498,20.429791
1,5.257844,7.104546,4.883344,5.856238
2,-41.016579,-6.723576,-18.682562,-27.106210
3,-59.055194,-4.241649,-43.887025,-39.791187
4,-68.597914,-11.924953,-50.747156,-47.750383


In [14]:
def round_money_only_hundred_k_tour(row):
    won = ['롯데', '신한', '우리', '하나', '현대', 'BC']
    card = row['카드사']
    brand = row['관광지']
    try:
        sales = float(row['매출액'])
    except ValueError as e:
        print(row)
        raise ValueError(e)
    abs_sales = abs(sales)
    
    if (card not in won) or (card == '하나' and brand == '전체 신판 매출'):
        return sales / 100
    
    if pd.isna(sales):
        result = 0
    else:
        result = sales / 1000000
    return result / 100

In [62]:
def get_tour(df, dates):
    tour = df.melt(id_vars=['카드사', '관광지'], value_vars=dates, var_name='기간', value_name='매출액')
    tour['매출액'] = tour['매출액'].str.strip()
    tour['매출액'] = tour['매출액'].str.replace(',','')
#     tour['매출액'] = tour['매출액'].str.replace('','0')
    tour['매출액_억원'] = tour.apply(round_money_only_hundred_k_tour, axis=1)
    tour['관광지'].replace(' 도쿄', '도쿄', inplace=True)
    tour['관광지'].replace(' 관광지 매출', '관광지 매출', inplace=True)
    tour['관광지'].replace(' 오사카', '오사카', inplace=True)
    tour['관광지'].replace('브랜드 합계', '관광지 매출', inplace=True)
    tour['관광지'].replace('관광지 합계', '관광지 매출', inplace=True)
    tour['관광지'].replace('전체신판매출', '전체 신판 매출', inplace=True)
    tour['관광지'].replace(' 오키나와', '오키나와', inplace=True)
    tour['관광지'].replace(' 후쿠오카', '후쿠오카', inplace=True)
    tour['관광지'].replace('상기합계', '관광지 매출', inplace=True)

    tour = tour.pivot_table(index=['카드사', '기간'], columns='관광지', values='매출액_억원')
    tour.reset_index(inplace=True)
    idx_hana = tour['카드사']=='하나'
    tour.loc[idx_hana, '관광지 매출'] = tour.loc[idx_hana, '도쿄'] + tour.loc[idx_hana, '오사카'] + tour.loc[idx_hana, '오키나와'] + tour.loc[idx_hana, '후쿠오카']
    
    tour['기간'] = pd.to_datetime(tour['기간'])

    return tour

# tour_2019

In [16]:
sheet_index=2
spread.open_sheet(sheet_index)
tmp_tour_2019 = spread.sheet_to_df(index=0, sheet=sheet_index)
tmp_tour_2019.head()

,카드사,관광지,20190624,20190701,20190708,20190715,20190722
0,KB국민,오사카,447,424,483,365,608
1,KB국민,도쿄,884,999,910,758,"1,232"
2,KB국민,후쿠오카,362,402,340,303,477
3,KB국민,오키나와,108,95,112,68,176
4,KB국민,전체신판매출,"3,031","3,284","3,071","2,684","4,389"


In [63]:
tour_2019 = get_tour(tmp_tour_2019, dates_2019)
tour_2019

관광지,카드사,기간,관광지 매출,도쿄,오사카,오키나와,전체 신판 매출,후쿠오카
0,BC,2019-06-24,31.550139,18.815526,6.429017,1.159040,751.453497,5.146555
1,BC,2019-07-01,30.891751,18.005142,6.457651,1.100907,820.073747,5.328052
2,BC,2019-07-08,27.996487,17.490291,5.353687,1.040409,770.949366,4.112101
3,BC,2019-07-15,24.284687,14.963331,4.983703,0.766388,744.027644,3.571266
4,BC,2019-07-22,23.777339,14.147724,4.766095,1.203205,718.111049,3.660315
5,KB국민,2019-06-24,18.010000,8.840000,4.470000,1.080000,30.310000,3.620000
6,KB국민,2019-07-01,19.210000,9.990000,4.240000,0.950000,32.840000,4.020000
7,KB국민,2019-07-08,18.450000,9.100000,4.830000,1.120000,30.710000,3.400000
8,KB국민,2019-07-15,14.930000,7.580000,3.650000,0.680000,26.840000,3.030000
9,KB국민,2019-07-22,24.940000,12.320000,6.080000,1.760000,43.890000,4.770000


In [82]:
def get_sum_tour(tour):
    cols_brand = ['관광지 매출', '도쿄', '오사카', '오키나와', '후쿠오카']
    summ = tour[['기간'] + cols_brand].groupby('기간').sum()
    summ.reset_index(inplace=True)
#     summ['기간'] = pd.to_datetime(summ['기간'])
    return summ


sum_tour_2019 = get_sum_tour(tour_2019)
sum_tour_2019

관광지,기간,관광지 매출,도쿄,오사카,오키나와,후쿠오카
0,2019-06-24,164.776533,86.734667,42.649755,6.826005,28.566106
1,2019-07-01,157.571232,89.086873,33.080677,6.662348,28.731334
2,2019-07-08,146.352454,81.141102,32.320788,6.829184,26.061380
3,2019-07-15,128.649319,74.842999,27.161001,5.826044,20.829274
4,2019-07-22,135.549543,75.961067,29.188156,7.698361,22.691959


# tour_2018

In [79]:
sheet_index=3
spread.open_sheet(sheet_index)
tmp_tour_2018 = spread.sheet_to_df(index=0, sheet=sheet_index)
tmp_tour_2018.head()

,카드사,관광지,20180624,20180701,20180708,20180715,20180722
0,KB국민,오사카,297,344,363,369,615
1,KB국민,도쿄,692,756,719,717,"1,072"
2,KB국민,후쿠오카,342,351,308,278,457
3,KB국민,오키나와,90,84,85,90,123
4,KB국민,전체신판매출,"2,384","2,521","2,421","2,460","3,868"


In [81]:
tour_2018 = get_tour(tmp_tour_2018, dates_2018)
tour_2018

관광지,카드사,기간,관광지 매출,도쿄,오사카,오키나와,전체 신판 매출,후쿠오카
0,BC,2018-06-24,26.376230,15.647552,5.528590,1.084613,662.479551,4.115476
1,BC,2018-07-01,28.484345,16.119412,7.220643,1.078415,724.415458,4.065875
2,BC,2018-07-08,26.415534,13.826716,7.755958,0.992749,689.556769,3.840111
3,BC,2018-07-15,26.069461,13.553036,7.702770,1.039280,682.338349,3.774376
4,BC,2018-07-22,26.849323,13.600460,7.998394,1.246665,657.344982,4.003803
5,KB국민,2018-06-24,14.210000,6.920000,2.970000,0.900000,23.840000,3.420000
6,KB국민,2018-07-01,15.340000,7.560000,3.440000,0.840000,25.210000,3.510000
7,KB국민,2018-07-08,14.740000,7.190000,3.630000,0.850000,24.210000,3.080000
8,KB국민,2018-07-15,14.550000,7.170000,3.690000,0.900000,24.600000,2.780000
9,KB국민,2018-07-22,22.670000,10.720000,6.150000,1.230000,38.680000,4.570000


In [84]:
sum_tour_2019

관광지,기간,관광지 매출,도쿄,오사카,오키나와,후쿠오카
0,2019-06-24,164.776533,86.734667,42.649755,6.826005,28.566106
1,2019-07-01,157.571232,89.086873,33.080677,6.662348,28.731334
2,2019-07-08,146.352454,81.141102,32.320788,6.829184,26.061380
3,2019-07-15,128.649319,74.842999,27.161001,5.826044,20.829274
4,2019-07-22,135.549543,75.961067,29.188156,7.698361,22.691959


In [80]:
tmp_tour_2018

,카드사,관광지,20180624,20180701,20180708,20180715,20180722
0,KB국민,오사카,297,344,363,369,615
1,KB국민,도쿄,692,756,719,717,"1,072"
2,KB국민,후쿠오카,342,351,308,278,457
3,KB국민,오키나와,90,84,85,90,123
4,KB국민,전체신판매출,"2,384","2,521","2,421","2,460","3,868"
5,KB국민,관광지 매출,"1,421","1,534","1,474","1,455","2,267"
6,롯데,오사카,"54,119,963","98,620,910","86,074,851","100,890,860","70,664,716"
7,롯데,도쿄,"146,659,072","147,705,570","154,709,768","118,593,993","135,316,391"
8,롯데,후쿠오카,"101,303,068","79,893,188","84,009,966","82,418,942","107,537,035"
9,롯데,오키나와,"12,938,093","18,794,754","21,174,323","29,836,913","21,646,019"


In [83]:
sum_tour_2018 = get_sum_tour(tour_2018)
sum_tour_2018

관광지,기간,관광지 매출,도쿄,오사카,오키나와,후쿠오카
0,2018-06-24,131.175909,73.518424,25.941168,6.994983,24.721334
1,2018-07-01,135.645715,75.513321,30.739088,6.441025,22.962281
2,2018-07-08,133.719334,69.893799,34.149867,6.712187,22.973480
3,2018-07-15,134.034565,68.614840,35.137523,7.293223,22.978979
4,2018-07-22,144.989082,73.859251,38.626108,7.719114,24.784609
